In [1]:
import tensorflow as tf
import math
from IPython import display
import numpy as np
import pandas as pd
from sklearn import metrics
from matplotlib import pyplot as plt
from matplotlib import gridspec
from matplotlib import cm
from tensorflow.python.data import Dataset
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format
print("Completed")

Completed


In [2]:
california_housing_dataframe = pd.read_csv('/Users/biplabkumardas/google-tensorflow/california_housing_train.csv', sep=",")
print("Completed")

Completed


In [3]:
california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe["median_house_value"] /= 1000.0
california_housing_dataframe

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
3360,-117.9,33.9,17.0,6100.0,861.0,2771.0,866.0,7.6,306.7
3890,-118.0,33.8,24.0,2027.0,358.0,1405.0,341.0,5.1,231.4
14684,-122.2,37.8,45.0,990.0,261.0,901.0,260.0,2.2,82.0
8049,-118.4,34.1,38.0,3830.0,518.0,1292.0,516.0,12.8,500.0
11600,-121.3,38.0,8.0,1868.0,392.0,1258.0,389.0,3.2,95.9
...,...,...,...,...,...,...,...,...,...
8407,-118.5,34.0,25.0,2689.0,719.0,1229.0,663.0,3.6,500.0
16847,-123.4,39.4,18.0,3946.0,813.0,1899.0,730.0,2.6,124.6
1436,-117.2,34.1,40.0,2374.0,500.0,1772.0,455.0,2.2,72.5
14079,-122.1,38.2,34.0,1562.0,289.0,898.0,307.0,3.4,107.2


In [4]:
california_housing_dataframe.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,207.3
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,116.0
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,15.0
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,119.4
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,180.4
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,265.0
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,500.0


In [5]:
## getting the feature and ground truth or the labels associated with 
my_feature = california_housing_dataframe[['total_rooms']]
feature_columns = [tf.feature_column.numeric_column("total_rooms")]
print(my_feature)
print(feature_columns)

       total_rooms
3360        6100.0
3890        2027.0
14684        990.0
8049        3830.0
11600       1868.0
...            ...
8407        2689.0
16847       3946.0
1436        2374.0
14079       1562.0
950         2183.0

[17000 rows x 1 columns]
[_NumericColumn(key='total_rooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [6]:
## define the labels
targets = california_housing_dataframe["median_house_value"]
print(targets)

3360    306.7
3890    231.4
14684    82.0
8049    500.0
11600    95.9
         ... 
8407    500.0
16847   124.6
1436     72.5
14079   107.2
950     169.7
Name: median_house_value, Length: 17000, dtype: float64


In [7]:
## define the optimizer
my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0000001)
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

linear_regressor = tf.estimator.LinearRegressor(
    feature_columns = feature_columns,
    optimizer = my_optimizer
)

print("Completed")

Completed


In [8]:
# preprocessing of the data
def my_input_fn(features, batch_size = 1, shuffle = True, num_epochs=None):
    features = {key:np.array(value) for key, value in dict(features).items()}
    ds = Dataset.from_tensor_slices((features, targets))
    ds = ds.batch(batch_size).repeat(num_epochs)
    if shuffle:
        ds = ds.shuffle(buffer_size=10000)
        features, labels = ds.make_one_shot_iterator().get_next()
        return features, labels

print("Completed")

Completed


In [33]:
linear_regressor.train(
    input_fn=lambda:my_input_fn(my_feature, targets),
    steps=100
)

TypeError: data type not understood

In [28]:
prediction_input_fn =lambda: my_input_fn(my_feature, targets, num_epochs=1, shuffle=False)

# Call predict() on the linear_regressor to make predictions.
predictions = linear_regressor.predict(input_fn=prediction_input_fn)

# Format predictions as a NumPy array, so we can calculate error metrics.
predictions = np.array([item['predictions'][0] for item in predictions])

# Print Mean Squared Error and Root Mean Squared Error.
mean_squared_error = metrics.mean_squared_error(predictions, targets)
root_mean_squared_error = math.sqrt(mean_squared_error)
print ("Mean Squared Error (on training data):" )
mean_squared_error
print ("Root Mean Squared Error (on training data): %0.3f")  
root_mean_squared_error

ValueError: Could not find trained model in model_dir: /var/folders/6n/2pdm794s2j52kx6p4v6whh4m0000gn/T/tmp5h58h9px.